<a href="https://colab.research.google.com/github/Nurslatip/Bishkek-Real-Estate-Price-Prediction/blob/main/1st_comp_rf_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.5/585.5 kB 36.5 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import re
warnings.simplefilter("ignore")

sns.set(style="darkgrid")
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_predict, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown="ignore")
from sklearn.ensemble import RandomForestRegressor
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

In [ ]:
from google.colab import files
files.upload()

In [ ]:
train_file_path ="/content/train.csv"
train = pd.read_csv(train_file_path)
test_file_path = "/content/test.csv"
test = pd.read_csv(test_file_path)

In [ ]:
test_id_column = test.id
cols_to_drop = [
    'Телефон',
    'Интернет',
    'Мебель',
    'Возможность рассрочки',
    'Возможность обмена',
    'Разное',
    'Пол',
    'Возможность ипотеки',
    'Площадь участка',
    'Канализация',
    'Питьевая вода',
    'Электричество',
    'added',
    'upped',
    'view_count',
    'hearts'
]
train.drop(columns=cols_to_drop, inplace=True)


test.drop(columns=cols_to_drop, inplace=True)
test.drop('id', axis=1, inplace=True)


train.insert(0, 'room_count', train['main'].str.extract(r'(\d+)-комн'))
train['room_count'] = train['room_count'].astype(float)


test.insert(0, 'room_count', test['main'].str.extract(r'(\d+)-комн'))
test['room_count'] = test['room_count'].astype(float)


train.loc[train.room_count.isna(), 'room_count'] = 7


test.loc[test.room_count.isna(), 'room_count'] = 7


train.drop(6674, axis=0, inplace=True)
train.drop(2255, axis=0, inplace=True)


train.insert(8, 'house_material', train['Дом'].str.extract(r'^(.*?)\s*(?:,|$)'))
train.insert(9, 'year_built', train['Дом'].str.extract(r'(\d{4})').astype(float))

test.insert(8, 'house_material', test['Дом'].str.extract(r'^(.*?)\s*(?:,|$)'))
test.insert(9, 'year_built', test['Дом'].str.extract(r'(\d{4})').astype(float))


def fill_year_built_by_series_mode(df, series_col='Серия', year_col='year_built'):
    mode_years = df.groupby(series_col)[year_col].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    global_mode = df[year_col].mode().iloc[0]

    def fill_row(row):
        if pd.isna(row[year_col]):
            return mode_years.get(row[series_col], global_mode)
        else:
            return row[year_col]

    df[year_col] = df.apply(fill_row, axis=1)
    return df


train = fill_year_built_by_series_mode(train)
test = fill_year_built_by_series_mode(test)


def add_is_new_building_feature(df, year_col='year_built', threshold_year=2025, insert_pos=10):
    is_new = df[year_col] >= threshold_year
    df['is_new_building'] = is_new.map({True: 'Yes', False: 'No'})
    col = df.pop('is_new_building')
    df.insert(insert_pos, 'is_new_building', col)
    return df


train = add_is_new_building_feature(train)
test = add_is_new_building_feature(test)


train.drop([5075, 5424], axis=0, inplace=True)


def extract_floor(s):
    if pd.isna(s):
        return None
    try:
        floor_num = int(s.split()[0])
        return floor_num
    except:
        return None


def extract_total_floors(s):
    if pd.isna(s):
        return None
    try:
        total = int(s.split()[-1])
        return total
    except:
        return None

train.insert(12, 'floor', train['Этаж'].apply(extract_floor))
train.insert(13, 'total_floors', train['Этаж'].apply(extract_total_floors))


test.insert(12, 'floor', test['Этаж'].apply(extract_floor))
test.insert(13, 'total_floors', test['Этаж'].apply(extract_total_floors))


def process_basement_feature(df, floor_col='Этаж', insert_pos=14):
    mask_basement = df[floor_col].str.contains(r'цоколь|подвал', case=False, na=False)

    df['is_basement'] = 'No'
    df.loc[mask_basement, 'is_basement'] = 'Yes'

    df.loc[mask_basement, 'floor'] = 0

    df.loc[mask_basement, 'total_floors'] = (
        df.loc[mask_basement, floor_col]
        .str.extract(r'из (\d+)')
        .iloc[:, 0]
        .astype(float)
    )

    col = df.pop('is_basement')
    df.insert(insert_pos, 'is_basement', col)

    return df


train = process_basement_feature(train)
test = process_basement_feature(test)


train.drop(6824, axis=0, inplace=True)


train.insert(16, 'total_area', (
    train['Площадь']
    .str.extract(r'^([\d.]+)')
    .iloc[:, 0]
    .astype(float)
))


test.insert(16, 'total_area', (
    test['Площадь']
    .str.extract(r'^([\d.]+)')
    .iloc[:, 0]
    .astype(float)
))


def add_missing_flag(df, col_name, insert_pos):
    missing_flag = df[col_name].isna().map({True: 'Yes', False: 'No'})
    df.insert(insert_pos, f'{col_name}_missing', missing_flag)
    df[col_name] = df[col_name].fillna('не указано')
    return df


In [ ]:


train = add_missing_flag(train, 'Отопление', 19)
train = add_missing_flag(train, 'Состояние', 20)

test = add_missing_flag(test, 'Отопление', 19)
test = add_missing_flag(test, 'Состояние', 20)


features_to_drop = [
    'main',
    'Дом',
    'Этаж',
    'Площадь',
    'Газ',
    'Санузел',
    'Балкон',
    'Входная дверь',
    'Парковка',
    'Высота потолков',
    'Безопасность',
    'Правоустанавливающие документы',
    'address',
    'is_new_building',
    'is_basement',
    'Отопление_missing',
    'Состояние_missing',
    'Тип предложения',
    'year_built'
]
train.drop([6582, 3538, 1535], axis=0, inplace=True)


train.drop(features_to_drop, axis=1, inplace=True)
test.drop(features_to_drop, axis=1, inplace=True)


In [ ]:
y = train.usd_price
train.drop('usd_price' , axis = 1 , inplace=True)
X = train


In [ ]:
numeric_features = train.select_dtypes([np.number])
categorical_features = train.dtypes[train.dtypes == "object"].index.tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=.2, random_state=42)

In [ ]:
def objective_rf(trial):
    # Hyperparameters for RandomForestRegressor
    n_estimators = trial.suggest_int("rf_regressor__n_estimators", 50, 500) # Number of trees
    max_depth = trial.suggest_int("rf_regressor__max_depth", 5, 25) # Max depth of trees
    min_samples_split = trial.suggest_int("rf_regressor__min_samples_split", 2, 20) # Min samples to split
    min_samples_leaf = trial.suggest_int("rf_regressor__min_samples_leaf", 1, 10) # Min samples per leaf
    # max_features is crucial for Random Forests to reduce correlation between trees
    max_features = trial.suggest_categorical("rf_regressor__max_features", ['sqrt', 'log2', 0.6, 0.8, 1.0])

    # Determine numeric and categorical features within the function, based on X_train
    numeric_features_local = X_train.select_dtypes([np.number]).columns.tolist()
    categorical_features_local = X_train.dtypes[X_train.dtypes == "object"].index.tolist()

    # ColumnTransformer setup
    column_transformer = ColumnTransformer([
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_local),
        ('scaler', MinMaxScaler(), numeric_features_local) # Include MinMaxScaler
    ], remainder='passthrough')

    # Create the pipeline with chosen hyperparameters
    pipeline = Pipeline([
        ('preprocessing', column_transformer),
        ('rf_regressor', RandomForestRegressor( # Changed to RandomForestRegressor
            random_state=42,
            n_jobs=-1, # Use all available CPU cores for faster training
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features
        ))
    ])

    # Evaluate with cross-validation
    y_pred_cv = cross_val_predict(pipeline, X_train, y_train, cv=5, n_jobs=-1)

    r2 = r2_score(y_train, y_pred_cv)
    return r2

# --- Optuna Study Setup and Run ---
study_rf = optuna.create_study(
    storage="sqlite:///rf_optuna_study.db", # Changed database name
    study_name="random_forest_hyperparameter_tuning", # Changed study name
    direction="maximize",
    load_if_exists=True,
    sampler=TPESampler(seed=42,),
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)

print("Starting Optuna optimization for RandomForestRegressor...")
study_rf.optimize(objective_rf, n_trials=100, timeout=600)

# --- 4. Get Best Results ---
print("\n--- Optuna Results for RandomForestRegressor ---")
print(f"Number of finished trials: {len(study_rf.trials)}")
print(f"Best trial:")
print(f"  Value (R2): {study_rf.best_value:.4f}")
print("  Params: ")
for key, value in study_rf.best_params.items():
    print(f"    {key}: {value}")

# --- 5. Train Final Model with Best Parameters ---
best_params_rf = study_rf.best_params

# Extract parameters for RandomForestRegressor from Optuna's best_params
# Note the prefix 'rf_regressor__'
rf_params = {k.replace('rf_regressor__', ''): v for k, v in best_params_rf.items()}

# Create ColumnTransformer (same as in objective)
# Determine numeric and categorical features based on the full training data before the split for the final model
numeric_features_final = X.select_dtypes([np.number]).columns.tolist()
categorical_features_final = X.dtypes[X.dtypes == "object"].index.tolist()

final_column_transformer = ColumnTransformer([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_final),
    ('scaler', MinMaxScaler(), numeric_features_final)
], remainder='passthrough')

# Create final pipeline
final_pipeline_rf = Pipeline([
    ('preprocessing', final_column_transformer),
    ('rf_regressor', RandomForestRegressor(
        random_state=42,
        n_jobs=-1, # Use all available CPU cores for final model too
        **rf_params # Unpack the best parameters
    ))
])

# Fit the final model on the entire training dataset
print("\nFitting final RandomForestRegressor pipeline on the entire training dataset...")
final_pipeline_rf.fit(X_train, y_train)

# Evaluate the final model on the training set with cross-validation
y_pred_train_final_rf = cross_val_predict(final_pipeline_rf, X_train, y_train, cv=5, n_jobs=-1)

print("\n--- Final RandomForestRegressor CV Metrics (using Optuna's best params) ---")
print("CV R2:\t\t %.4f" % r2_score(y_train, y_pred_train_final_rf))
print("CV MAE:\t\t %.4f" % mean_absolute_error(y_train, y_pred_train_final_rf))
print("CV MAPE:\t %.4f" % mean_absolute_percentage_error(y_train, y_pred_train_final_rf))
print("CV MSE:\t\t %.4f" % mean_squared_error(y_train, y_pred_train_final_rf))

# Evaluate on the test set
y_pred_test_final_rf = final_pipeline_rf.predict(X_test)
print("\n--- Final RandomForestRegressor Test Set Metrics ---")
print("Test R2:\t\t %.4f" % r2_score(y_test, y_pred_test_final_rf))
print("Test MAE:\t\t %.4f" % mean_absolute_error(y_test, y_pred_test_final_rf))

[I 2025-07-31 14:13:12,796] Using an existing study with name 'random_forest_hyperparameter_tuning' instead of creating a new one.


Starting Optuna optimization for RandomForestRegressor...


[I 2025-07-31 14:13:28,302] Trial 2 finished with value: 0.8782732835020566 and parameters: {'rf_regressor__n_estimators': 218, 'rf_regressor__max_depth': 24, 'rf_regressor__min_samples_split': 15, 'rf_regressor__min_samples_leaf': 6, 'rf_regressor__max_features': 0.8}. Best is trial 2 with value: 0.8782732835020566.
[I 2025-07-31 14:13:43,000] Trial 3 finished with value: 0.8489861659302265 and parameters: {'rf_regressor__n_estimators': 369, 'rf_regressor__max_depth': 5, 'rf_regressor__min_samples_split': 20, 'rf_regressor__min_samples_leaf': 9, 'rf_regressor__max_features': 1.0}. Best is trial 2 with value: 0.8782732835020566.
[I 2025-07-31 14:13:58,645] Trial 4 finished with value: 0.881263275381596 and parameters: {'rf_regressor__n_estimators': 244, 'rf_regressor__max_depth': 11, 'rf_regressor__min_samples_split': 13, 'rf_regressor__min_samples_leaf': 2, 'rf_regressor__max_features': 0.8}. Best is trial 4 with value: 0.881263275381596.
[I 2025-07-31 14:14:15,563] Trial 5 finished w


--- Optuna Results for RandomForestRegressor ---
Number of finished trials: 33
Best trial:
  Value (R2): 0.8902
  Params: 
    rf_regressor__n_estimators: 411
    rf_regressor__max_depth: 17
    rf_regressor__min_samples_split: 4
    rf_regressor__min_samples_leaf: 2
    rf_regressor__max_features: 0.6

Fitting final RandomForestRegressor pipeline on the entire training dataset...

--- Final RandomForestRegressor CV Metrics (using Optuna's best params) ---
CV R2:		 0.8902
CV MAE:		 12969.9524
CV MAPE:	 0.1052
CV MSE:		 636764156.5493

--- Final RandomForestRegressor Test Set Metrics ---
Test R2:		 0.9257
Test MAE:		 11635.5562


In [ ]:
rf_optuna = final_pipeline_rf.predict(test)


In [ ]:
submission_df = pd.DataFrame({
    'id': test_id_column,
    'predicted_price': rf_optuna
})

In [ ]:
submission_df.to_csv('dt_rfr.csv', index=False)

In [ ]:
def objective_rf_mape(trial):
    n_estimators = trial.suggest_int("rf_regressor__n_estimators", 50, 500)
    max_depth = trial.suggest_int("rf_regressor__max_depth", 5, 25)
    min_samples_split = trial.suggest_int("rf_regressor__min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("rf_regressor__min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("rf_regressor__max_features", ['sqrt', 'log2', 0.6, 0.8, 1.0])

    # Determine numeric and categorical features within the function, based on X_train
    numeric_features_local_trial = X_train.select_dtypes([np.number]).columns.tolist()
    categorical_features_local_trial = X_train.dtypes[X_train.dtypes == "object"].index.tolist()

    column_transformer = ColumnTransformer([
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_local_trial),
        ('scaler', MinMaxScaler(), numeric_features_local_trial)
    ], remainder='passthrough')

    pipeline = Pipeline([
        ('preprocessing', column_transformer),
        ('rf_regressor', RandomForestRegressor(
            random_state=42,
            n_jobs=-1,
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features
        ))
    ])

    y_pred_cv = cross_val_predict(pipeline, X_train, y_train, cv=5, n_jobs=-1)

    # Calculate MAPE. Ensure y_train does not have zeros.
    # The y_train passed to cross_val_predict (and thus here) already has epsilon added if needed.
    mape = mean_absolute_percentage_error(y_train, y_pred_cv)
    return mape # Optuna will MINIMIZE this value


# --- Optuna Study Setup and Run (Optimizing for MAPE) ---
study_rf_mape = optuna.create_study(
    storage="sqlite:///rf_optuna_mape_study.db", # New database name for MAPE optimization
    study_name="random_forest_mape_tuning", # New study name
    direction="minimize", # <--- CRITICAL CHANGE: Minimize MAPE
    load_if_exists=True,
    sampler=TPESampler(seed=42,),
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)

print("Starting Optuna optimization for RandomForestRegressor, focusing on MINIMIZING MAPE...")
study_rf_mape.optimize(objective_rf_mape, n_trials=100, timeout=600)

# --- 4. Get Best Results ---
print("\n--- Optuna Results for RandomForestRegressor (MAPE Optimization) ---")
print(f"Number of finished trials: {len(study_rf_mape.trials)}")
print(f"Best trial:")
# Optuna returns the optimized metric value, which is MAPE in this case
print(f"  Value (MAPE): {study_rf_mape.best_value:.4f}")
print("  Params: ")
for key, value in study_rf_mape.best_params.items():
    print(f"    {key}: {value}")

# --- 5. Train Final Model with Best Parameters ---
best_params_rf_mape = study_rf_mape.best_params

rf_params_mape = {k.replace('rf_regressor__', ''): v for k, v in best_params_rf_mape.items()}

# Define features for the final pipeline
numeric_features_final = X_train.select_dtypes([np.number]).columns.tolist()
categorical_features_final = X_train.dtypes[X_train.dtypes == "object"].index.tolist()


final_column_transformer_mape = ColumnTransformer([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_final),
    ('scaler', MinMaxScaler(), numeric_features_final)
], remainder='passthrough')

final_pipeline_rf_mape = Pipeline([
    ('preprocessing', final_column_transformer_mape),
    ('rf_regressor', RandomForestRegressor(
        random_state=42,
        n_jobs=-1,
        **rf_params_mape
    ))
])

print("\nFitting final RandomForestRegressor pipeline (MAPE optimized) on the entire training dataset...")
final_pipeline_rf_mape.fit(X_train, y_train) # Fit using the y_train that was used for Optuna (potentially adjusted for MAPE)

# --- Evaluate with cross-validation on Training Set ---
y_pred_train_final_rf_mape = cross_val_predict(final_pipeline_rf_mape, X_train, y_train, cv=5, n_jobs=-1)

print("\n--- Final RandomForestRegressor CV Metrics (MAPE Optimized) ---")
print("CV R2:\t\t %.4f" % r2_score(y_train, y_pred_train_final_rf_mape))
print("CV MAE:\t\t %.4f" % mean_absolute_error(y_train, y_pred_train_final_rf_mape))
print("CV MAPE:\t %.4f" % mean_absolute_percentage_error(y_train, y_pred_train_final_rf_mape)) # Optimized metric
print("CV MSE:\t\t %.4f" % mean_squared_error(y_train, y_pred_train_final_rf_mape))

# --- Evaluate on the Test Set ---
y_pred_test_final_rf_mape = final_pipeline_rf_mape.predict(X_test)
print("\n--- Final RandomForestRegressor Test Set Metrics (MAPE Optimized) ---")
print("Test R2:\t\t %.4f" % r2_score(y_test, y_pred_test_final_rf_mape))
print("Test MAE:\t\t %.4f" % mean_absolute_error(y_test, y_pred_test_final_rf_mape))
print("Test MAPE:\t\t %.4f" % mean_absolute_percentage_error(y_test, y_pred_test_final_rf_mape)) # Optimized metric
print("Test MSE:\t\t %.4f" % mean_squared_error(y_test, y_pred_test_final_rf_mape))

[I 2025-07-31 15:40:44,059] Using an existing study with name 'random_forest_mape_tuning' instead of creating a new one.


Starting Optuna optimization for RandomForestRegressor, focusing on MINIMIZING MAPE...


[I 2025-07-31 15:41:41,763] Trial 95 finished with value: 0.10381407390187342 and parameters: {'rf_regressor__n_estimators': 438, 'rf_regressor__max_depth': 25, 'rf_regressor__min_samples_split': 3, 'rf_regressor__min_samples_leaf': 1, 'rf_regressor__max_features': 0.8}. Best is trial 93 with value: 0.10375905711017382.
[I 2025-07-31 15:42:25,478] Trial 96 finished with value: 0.10383459613119751 and parameters: {'rf_regressor__n_estimators': 442, 'rf_regressor__max_depth': 25, 'rf_regressor__min_samples_split': 3, 'rf_regressor__min_samples_leaf': 1, 'rf_regressor__max_features': 0.8}. Best is trial 93 with value: 0.10375905711017382.
[I 2025-07-31 15:42:59,833] Trial 97 finished with value: 0.10772128690070945 and parameters: {'rf_regressor__n_estimators': 439, 'rf_regressor__max_depth': 25, 'rf_regressor__min_samples_split': 10, 'rf_regressor__min_samples_leaf': 2, 'rf_regressor__max_features': 0.8}. Best is trial 93 with value: 0.10375905711017382.
[I 2025-07-31 15:43:42,763] Trial


--- Optuna Results for RandomForestRegressor (MAPE Optimization) ---
Number of finished trials: 112
Best trial:
  Value (MAPE): 0.1036
  Params: 
    rf_regressor__n_estimators: 426
    rf_regressor__max_depth: 24
    rf_regressor__min_samples_split: 3
    rf_regressor__min_samples_leaf: 1
    rf_regressor__max_features: 0.6

Fitting final RandomForestRegressor pipeline (MAPE optimized) on the entire training dataset...

--- Final RandomForestRegressor CV Metrics (MAPE Optimized) ---
CV R2:		 0.8908
CV MAE:		 12778.3328
CV MAPE:	 0.1036
CV MSE:		 633103199.1219

--- Final RandomForestRegressor Test Set Metrics (MAPE Optimized) ---
Test R2:		 0.9261
Test MAE:		 11480.8509
Test MAPE:		 0.0965
Test MSE:		 419305005.8861


In [ ]:
rf_optunam = final_pipeline_rf_mape.predict(test)


In [ ]:
submission_df = pd.DataFrame({
    'id': test_id_column,
    'predicted_price': rf_optunam
})

In [ ]:
submission_df.to_csv('dt_rfr1.csv', index=False)